&emsp; 2 &ensp; CoLLiE 的使用案例

&emsp; &emsp; 2.1 &ensp; 案例：指令微调 MOSS

&emsp; &emsp; 2.2 &ensp; CoLLiE 的 Config 模块

&emsp; &emsp; 2.3 &ensp; CoLLiE 的 Trainer 模块

&emsp; &emsp; 2.4 &ensp; CoLLiE 的 Metric 模块

&emsp; &emsp; 2.5 &ensp; CoLLiE 的 Monitor 模块

## 2. &ensp; CoLLiE 的使用案例

### 2.1 &ensp; 案例：指令微调 MOSS

&ensp; &ensp; 在上一章中，我们已经介绍了 CoLLiE 提出的背景、实现的功能 以及 包含的模块。

&ensp; &ensp; 写个LOMO、ZeRO3、微调MOSS

In [6]:
!CUDA_VISIBLE_DEVICES=0,1,2,3 torchrun --nproc_per_node=4 --standalone example-tutorial-2-one-sentence-overfitting.py

*****************************************
Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
*****************************************
[2023-06-30 20:46:28,132] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[2023-06-30 20:46:28,133] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[2023-06-30 20:46:28,135] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[2023-06-30 20:46:28,136] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)

===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: ht

[2023-06-30 20:52:28,616] [INFO] [utils.py:785:see_memory_usage] DeepSpeedZeRoOffload initialize [begin]
[2023-06-30 20:52:28,616] [INFO] [utils.py:786:see_memory_usage] MA 7.93 GB         Max_MA 11.3 GB         CA 13.06 GB         Max_CA 13 GB 
[2023-06-30 20:52:28,617] [INFO] [utils.py:793:see_memory_usage] CPU Virtual Memory:  used = 19.89 GB, percent = 7.9%
Parameter Offload: Total persistent parameters: 1474560 in 138 params
[2023-06-30 20:52:28,736] [INFO] [utils.py:785:see_memory_usage] DeepSpeedZeRoOffload initialize [end]
[2023-06-30 20:52:28,737] [INFO] [utils.py:786:see_memory_usage] MA 7.93 GB         Max_MA 7.93 GB         CA 13.06 GB         Max_CA 13 GB 
[2023-06-30 20:52:28,737] [INFO] [utils.py:793:see_memory_usage] CPU Virtual Memory:  used = 19.89 GB, percent = 7.9%
[2023-06-30 20:52:28,738] [INFO] [config.py:960:print] DeepSpeedEngine configuration:
[2023-06-30 20:52:28,738] [INFO] [config.py:964:print]   activation_checkpointing_config  {
    "partition_activations

Training Epoch     0% ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ -- it./s 0:00:00 / -:--:-- 
Training Epoch     0% ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ -- it./s 0:00:00 / -:--:-- 
Training Epoch: 0 / 1    0% ━━━━━━━━━━━━━━━━━━━━━━━ -- it./s 0:00:00 / -:--:-- 
Training Epoch: 0 / 1    0% ━━━━━━━━━━━━━━━━━━━━━━━ -- it./s 0:00:00 / -:--:-- 
Training Batch: 0 / 13   0% ━━━━━━━━━━━━━━━━━━━━━━━ -- it./s 0:00:00 / -:--:-- Gradient overflow, skipping step 0
Gradient overflow, skipping step 0
Gradient overflow, skipping step 0
[LOMO] OVERFLOW! Rank 0 Skipping step. Attempted loss scale: 4294967296, 
reducing to 2147483648
Training Epoch: 0 / 1    0% ━━━━━━━━━━━━━━━━━━━━━━━ -- it./s 0:00:00 / -:--:-- 
Gradient overflow, skipping step 0━━━━━━━━━━━━━━━━━━━ -- it./s 0:00:00 / -:--:-- 
Training Epoch: 0 / 1    0% ━━━━━━━━━━━━━━━━━━━━━━━ -- it./s 0:00:00 / -:--:-- 
Training Epoch: 0 / 1    0% ━━━━━━━━━━━━ -- it./s 0:00:15 / -:--:--             
Training Epoch: 0 / 1    0% ━━━━━━━━━━━━ -- it./s 0:00:15 / -:--:--         

Training Batch: 7 / 13  62% ━━━━━━╺━━━ 0.14 it./s 0:01:07 / 0:00:37 Loss: 5.8789Gradient overflow, skipping step 8
Gradient overflow, skipping step 8
Gradient overflow, skipping step 8
[LOMO] OVERFLOW! Rank 0 Skipping step. Attempted loss scale: 16777216, reducing 
to 8388608
Training Epoch: 0 / 1    0% ━━━━━━━━━━ -- it./s   0:01:07 / -:--:--             
Gradient overflow, skipping step 8━━╺━━━ 0.14 it./s 0:01:07 / 0:00:37 Loss: 5.8789
Training Epoch: 0 / 1    0% ━━━━━━━━━━ -- it./s   0:01:07 / -:--:--             
Training Epoch: 0 / 1    0% ━━━━━━━━━━ -- it./s   0:01:14 / -:--:--             5.8789
Training Epoch: 0 / 1    0% ━━━━━━━━━━ -- it./s   0:01:14 / -:--:--             5.8789
Training Batch: 8 / 13  69% ━━━━━━╸━━━ 0.14 it./s 0:01:14 / 0:00:30 Loss: 5.8789Gradient overflow, skipping step 9Gradient overflow, skipping step 9

Gradient overflow, skipping step 9
[LOMO] OVERFLOW! Rank 0 Skipping step. Attempted loss scale: 8388608, reducing 
to 4194304
Training Epoch: 0 / 1    0% 

Training Epoch: 0 / 1     0% ━━━━━━━━━━ -- it./s  0:01:58 / -:--:--             
Training Batch: 12 / 13 100% ━━━━━━━━━━ 0.14      0:01:44 / 0:00:00 Loss: 5.8906
                                        it./s                                   
Evaluating Batch: 3 / 8  50% ━━━━━╺━━━━ 0.28      0:00:14 / 0:00:15             
Training Epoch: 0 / 1     0% ━━━━━━━━━━ -- it./s  0:02:01 / -:--:--             
Training Batch: 12 / 13 100% ━━━━━━━━━━ 0.14      0:01:44 / 0:00:00 Loss: 5.8906
                                        it./s                                   
Evaluating Batch: 4 / 8  62% ━━━━━━╺━━━ 0.28      0:00:17 / 0:00:11             
Training Epoch: 0 / 1     0% ━━━━━━━━━━ -- it./s  0:02:05 / -:--:--             
Training Batch: 12 / 13 100% ━━━━━━━━━━ 0.14      0:01:44 / 0:00:00 Loss: 5.8906
                                        it./s                                   
Evaluating Batch: 5 / 8  75% ━━━━━━━╸━━ 0.28      0:00:21 / 0:00:08             
Training Epoch: 0 / 1     0%

[20:55:45] INFO     [' It provides a simple and efficient', ' It    ]8;id=471029;file:///remote-home/xrliu/anaconda3/envs/collie/lib/python3.9/site-packages/collie-1.0.1-py3.9.egg/collie/metrics/decode.py\decode.py]8;;\:]8;id=617889;file:///remote-home/xrliu/anaconda3/envs/collie/lib/python3.9/site-packages/collie-1.0.1-py3.9.egg/collie/metrics/decode.py#51\51]8;;\
                    provides a simple and efficient', ' It provides             
                    a simple and efficient', ' It provides a simple             
                    and efficient']                                             
Training Epoch: 0 / 1     0% ━━━━━━━━━ -- it./s   0:02:55 / -:--:--             
Training Batch: 12 / 13 100% ━━━━━━━━━ 0.14 it./s 0:01:44 / 0:00:00 Loss: 5.8906
Evaluating Batch: 1 / 8  25% ━━╺━━━━━━ 21.21      0:00:42 / 0:02:08             
Training Epoch: 0 / 1     0% ━━━━━━━━━━ -- it./s  0:03:16 / -:--:--             
Training Batch: 12 / 13 100% ━━━━━━━━━━ 0.14      0:01:44

[20:57:31] INFO     [' It provides a simple and efficient', ' It    ]8;id=562275;file:///remote-home/xrliu/anaconda3/envs/collie/lib/python3.9/site-packages/collie-1.0.1-py3.9.egg/collie/metrics/decode.py\decode.py]8;;\:]8;id=130889;file:///remote-home/xrliu/anaconda3/envs/collie/lib/python3.9/site-packages/collie-1.0.1-py3.9.egg/collie/metrics/decode.py#51\51]8;;\
                    provides a simple and efficient', ' It provides             
                    a simple and efficient', ' It provides a simple             
                    and efficient']                                             
Training Epoch: 0 / 1     0% ━━━━━━━━━ -- it./s   0:04:41 / -:--:--             
Training Batch: 12 / 13 100% ━━━━━━━━━ 0.14 it./s 0:01:44 / 0:00:00 Loss: 5.8906
Evaluating Batch: 6 / 8  88% ━━━━━━━╸━ 21.29      0:02:29 / 0:00:22             
                                       s/it.                                    [20:57:31] WARNING  Metric:decode returns None when gettin

### 2.2 &ensp; CoLLiE 的 Config 模块

### 2.3 &ensp; CoLLiE 的 Trainer 模块

### 2.4 &ensp; CoLLiE 的 Metric 模块

### 2.5 &ensp; CoLLiE 的 Monitor 模块